In [2]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [3]:
path = "/Users/aryagupta/Desktop/yt-sentiment-analysis/youtube-comments-sentiment-analysis/data/processed/preprocessed_v1.csv"

data = pd.read_csv(path).dropna(subset=['clean_comment'])

In [4]:
data.shape

(36793, 2)

In [5]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
mlflow.set_experiment("Experiment 4.0 - Handling Imbalanced Data")

2025/05/08 10:50:55 INFO mlflow.tracking.fluent: Experiment with name 'Experiment 4.0 - Handling Imbalanced Data' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/725100858814278742', creation_time=1746681655782, experiment_id='725100858814278742', last_update_time=1746681655782, lifecycle_stage='active', name='Experiment 4.0 - Handling Imbalanced Data', tags={}>

In [8]:
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  
    max_features = 10000

    X_train, X_test, y_train, y_test = train_test_split(data['clean_comment'], data['category'], test_size=0.2, random_state=1, stratify=data['category'])

    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)  
    X_test_vec = vectorizer.transform(X_test)  

    if imbalance_method == 'class_weights':
        class_weight = 'balanced'
    else:
        class_weight = None  

        if imbalance_method == 'SMOTE':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
            
        elif imbalance_method == 'ADASYN':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
            
        elif imbalance_method == 'Undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
            
        elif imbalance_method == 'SMOTEENN':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        y_pred = model.predict(X_test_vec)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

imbalance_methods = ['class_weights', 'SMOTE', 'ADASYN', 'Undersampling', 'SMOTEENN']

for method in imbalance_methods:
    run_imbalanced_experiment(method)

2025/05/08 10:51:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/725100858814278742/runs/00e9f108c9d94c4f970f8e05ebbf4dd1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/725100858814278742


2025/05/08 10:52:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_SMOTE_RandomForest_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/725100858814278742/runs/5ff1ba507101495899aedd8d4ff5c553
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/725100858814278742


2025/05/08 10:52:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_ADASYN_RandomForest_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/725100858814278742/runs/b55c854534ee423995fc3aed246dd955
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/725100858814278742


2025/05/08 10:52:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_Undersampling_RandomForest_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/725100858814278742/runs/3c09d404ce174c4b905cef9a7c7fca85
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/725100858814278742


2025/05/08 10:53:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_SMOTEENN_RandomForest_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/725100858814278742/runs/6d46bf6bcf7c4db5bb5ad00fffe50aa1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/725100858814278742
